# Land Cover Change Detection in Austin, TX
In this notebook, we'll learn how to apply a land cover change detection model to Planet imagery captured over the greater Austin area. 
We will use the following process to achieve our goal:
   1. Initialize a Dask CUDA cluster to scale predictions on a GPU
   2. Load and pre-process two rasters that cover the greater Austin area (one from 2017 and the other from 2022)
   3. Apply a pre-trained PyTorch change detection model to chunks of the scene in parallel.

## Load Packages and Initialize Dask CUDA Cluster

In [2]:
import utils
import torch
from models.basic_model import CDEvaluator
import os
import numpy as np
import pandas as pd
from typing import TypedDict
import xarray as xr
import rioxarray as rxr
import rasterio as rio
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=4)
client = Client(cluster)
client

2022-06-09 16:11:03,199 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 83.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44435,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 83.59 GiB
Comm: tcp://127.0.0.1:32891,Total threads: 4
Dashboard: http://127.0.0.1:41855/status,Memory: 83.59 GiB
Nanny: tcp://127.0.0.1:33651,


## Load pre-trained change detection model
The following cells load the pre-trained change detection model and sends the model parameters to the GPU to hold for processing.

In [ ]:
args = utils.get_args()
utils.get_device(args)

In [ ]:
def load_model():
    args.checkpoint_dir = os.path.join(args.checkpoint_root, args.project_name)
    model = CDEvaluator(args)
    model.load_checkpoint(args.checkpoint_name) 
    model.eval()
    return model
remote_model = client.submit(load_model)
print(remote_model)

## Load and pre-process Planet data
The following cell loads two rasters, one from May 2017 and the other from May 2022, and pre-processes them so that they can be passed into the change detection model. This data contains RGB and infrared, so we first remove the infrared band. We normalize each raster by subtracting the mean and dividing by the standard deviation. This helps the model by scaling the data into a range that the model has been trained on. In case the shape of the combined rasters won't produce equally-sized chunks, we cut off the last few pixels and only keep chunks that are of compatible size for loading onto the GPU. The combined dataset, **ds_comb** (2 years, 3 bands, x, y), holds all the data we are interested in - 2 years of RGB data over the area of interest. 

In [ ]:
a = rio.vrt.WarpedVRT(rio.open('./catalogs/17_05/mosaic.tif'))
b = rio.vrt.WarpedVRT(rio.open('./catalogs/22_05/mosaic.tif'),transform=a.transform,height=a.height,width=a.width)

ds1 = rxr.open_rasterio(a,chunks=(4,8192,8192),lock=False)
ds2 = rxr.open_rasterio(b,chunks=(4,8192,8192),lock=False)

ds1 = ds1[:3]
ds2 = ds2[:3]

ds1 = ds1/255.0
ds2 = ds2/255.0

m1 = ds1.mean(axis=[1,2])
s1 = ds1.std(axis=[1,2])
m2 = ds2.mean(axis=[1,2])
s2 = ds2.std(axis=[1,2])

ds = xr.combine_nested([ds1,ds2],concat_dim="time")

bands = xr.DataArray([1,2,3],name="band",dims=["band"],coords={"band":[1,2,3]})

first_mu = xr.DataArray(m1.data,name="mean",coords=[bands])
first_std = xr.DataArray(s1.data,name="std",coords=[bands])
second_mu = xr.DataArray(m2.data,name="mean",coords=[bands])
second_std = xr.DataArray(s2.data,name="std",coords=[bands])

mean = xr.concat([first_mu,second_mu],dim="time")
std = xr.concat([first_std,second_std],dim="time")

normalized = (ds-mean)/std

slices = {}
for coord in ["y","x"]:
    remainder = len(ds.coords[coord])%32
    slice_ = slice(-remainder) if remainder else slice(None)
    slices[coord] = slice_

ds_comb = normalized.isel(**slices)

ds_comb = ds_comb.chunk((2,3,8192,8192))
ds_comb

## Predict geographic changes for entire scene
Given normalized data, we can now pass this data into our pre-trained model. For each Dask chunk, we will split it into small chips and pass to the GPU for prediction. Our output is a binary mask for each chip. We stack each of these chips together to make a single mask for the entire scene, **predictions**.

In [ ]:
import dask.array
from itertools import product
def predict_chips(data,model)->torch.Tensor:
    result = model._forward_pass(data).cpu().numpy()[0][0]
    return result

def copy_and_predict_chunked(tile,model,token=None):
    out = np.empty(shape=tile.shape[2:], dtype="uint8")
    device = torch.device("cuda")
    w,h = out.shape
    grid = product(range(0, h-h%256, 256), range(0, w-w%256, 256))
    for i,j in grid:
        A = torch.as_tensor(tile[0][np.newaxis,:,j:j+256,i:i+256])
        B = torch.as_tensor(tile[1][np.newaxis,:,j:j+256,i:i+256])
        gpu_chip = {'name':'test',
                    'A':A.float().to(device),
                    'B':B.float().to(device),
                    'L':torch.zeros(1,1,256,256).float().to(device)
               }
        out[j:j+256,i:i+256] = predict_chips(gpu_chip,model)
    return out

meta = np.array([[]], dtype="uint8")[:0]

predictions_array = ds_comb.data.map_blocks(
    copy_and_predict_chunked,
    meta=meta,
    drop_axis=[0,1],
    model=remote_model,
    name="predict",
)

predictions = xr.DataArray(
    predictions_array,
    coords=ds_comb.drop_vars("band").coords,
    dims=("y", "x"),
)
predictions

## Visualize some of the results!
The entire dataset contains ~ 140 km x 120 km of data. This takes several minutes to process. Let's check out a subset of the data before we try to run the entire scene.

In [ ]:
import matplotlib.colors
from bokeh.models.tools import BoxZoomTool
import panel
import hvplot.xarray

def logo(plot, element):
    plot.state.toolbar.logo = None


zoom = BoxZoomTool(match_aspect=True)
style_kwargs = dict(
    width=450,
    height=400,
    xaxis=False,
    yaxis=False,
)
kwargs = dict(
    x="x",
    y="y",
    rasterize=True,
    cmap='gray',
    aggregator="min",
    colorbar=False,
    tools=["pan", zoom, "wheel_zoom", "reset"],
)

In [ ]:
middle = ds.shape[2] // 2, ds.shape[3] // 2
slice_y = slice(middle[0], middle[0] + 5_000)
slice_x = slice(middle[1], middle[1] + 5_000)

parts = [x.isel(y=slice_y, x=slice_x) for x in [ds, predictions]]

In [ ]:
ds_local, predictions_local = dask.compute(*parts)

In [ ]:
panel.Column(
    panel.Row(
        ds_local.sel(time=0)
        .hvplot.rgb(
            bands="band", rasterize=True, hover=False, title="Austin 05_17", tools=["pan", zoom, "wheel_zoom", "reset"], **style_kwargs
        )
        .opts(default_tools=[], hooks=[logo]),
        predictions_local
        .hvplot.image(title="Changes", **kwargs, **style_kwargs)
        .opts(default_tools=[]),
        ds_local.sel(time=1)
        .hvplot.rgb(
            bands="band",
            rasterize=True,
            hover=False,
            title="Austin 05_22",
            tools=["pan", zoom, "wheel_zoom", "reset"],
            **style_kwargs,
        )
        .opts(default_tools=[], hooks=[logo]),
    ),
)

## Analyze the change mask
We want to analyze the areas of change, so let's *polygonize* the mask so that it's easier to deal with for further tasks. We will also find the centroid of areas with the largest contiguous changes and pass those into a pre-trained classifier to look at changes in outputs.

In [5]:
from fastai.vision import *
from xrspatial.experimental import polygonize as polygonize

In [22]:
predictions_all = rxr.open_rasterio("change_map.tif",lock=False)
predictions_all = np.squeeze(predictions_all,axis=0)

In [23]:
gdf = polygonize(predictions_all,connectivity=8,return_type="geopandas")
gdf = gdf.assign(area=lambda df: df['geometry'].area,
                x=lambda df: df['geometry'].centroid.x.astype(int),
                y=lambda df: df['geometry'].centroid.y.astype(int),).sort_values(by='area',ascending=False)
gdf = gdf[1:10]

2022-06-09 16:20:28,439 - distributed.nanny - WARNING - Restarting worker
2022-06-09 16:20:29,780 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


KeyboardInterrupt: 

In [ ]:
learner = load_learner('../2750/')

In [ ]:
ims_17 = []
ims_22 = []
changes = []
for index,row in gdf.iterrows():
    ims_17.append(ds_local[0,:3,int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])
    ims_22.append(ds_local[1,:3,int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])
    changes.append(predictions_local[int(round(row['centroid-0']))-128:int(round(row['centroid-0']))+128,int(round(row['centroid-1']))-128:int(round(row['centroid-1']))+128])

In [ ]:
pred_17 = []
conf_17 = []
pred_22 = []
conf_22 = []
for im in range(len(images['ims_2017'])):
    if not im%50:
        print(f"predicting image {im}")
    temp_2017 = np.resize(images['ims_2017'][im].data,(3,224,224))
    temp_2022 = np.resize(images['ims_2022'][im].data,(3,224,224))
    temp_2017 = Image(torch.from_numpy(temp_2017).float())
    temp_2022 = Image(torch.from_numpy(temp_2022).float())
    p_17 = learner.predict(temp_2017)
    p_22 = learner.predict(temp_2022)
    pred_17.append(p_17[1].item())
    conf_17.append(p_17[2][p_17[1].item()].item())
    pred_22.append(p_22[1].item())
    conf_22.append(p_22[2][p_22[1].item()].item())

## Store out entire mask as tif

In [ ]:
#predictions.rio.to_raster("change_map.tif")